# **Week 12 - Neural Network**

The model uses an Adam optimizer for training efficiency through user-defined activations in hidden layers while implementing linear activation at its output section for regression tasks. MSE serves as the training and evaluation criterion when compiling the model.

# libraries

In [1]:
# Necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input # Use Input Layer explicitly
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error # To calculate MSE on original scale
import time
import os

# Seed

In [2]:
# Random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Global Variables

In [3]:
DATA_SIZES = [1000, 10000, 100000]
# File naming convention updated
FILE_NAME_TEMPLATE = "synthetic_data_{}k.csv"
# Feature and Target column names updated
FEATURE_COLUMNS = ['VarA', 'VarB', 'VarC', 'VarD']
TARGET_COLUMN = 'Target'

EPOCHS = 50
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.3
RANDOM_STATE = 42

# List to store results from all models and sizes
dl_results = []

# Functions

In [4]:
def build_model_simple(n_features, layer_nodes, model_name, activation='relu'):
    """Builds simple Keras Sequential models."""
    model = Sequential(name=model_name)
    model.add(Input(shape=(n_features,))) # Use Input layer
    for nodes in layer_nodes:
        model.add(Dense(nodes, activation=activation))
    model.add(Dense(1, activation='linear')) # Linear output for regression
    model.compile(optimizer='adam',
                  loss='mean_squared_error', # Use MSE for loss
                  metrics=['mean_squared_error']) # Track MSE metric
    return model


# Train and test

In [9]:
for size in DATA_SIZES:
    print(f"\n Data Size: {size}")
    data_file = FILE_NAME_TEMPLATE.format(size // 1000) # Use new file name format

    # Data Loading and Preparation
    if not os.path.exists(data_file):
        print(f"ERROR: Data file {data_file} not found. Please generate it first.")
        # Add placeholder results for the two remaining configurations for this size
        configs_to_skip = [
            '1 hidden layer 4 nodes',
            '2 hidden layers 4 nodes each'
            # Model 3 removed
        ]
        for config_name in configs_to_skip:
             dl_results.append({
                'Data size': size,
                'Configuration': config_name,
                'Training error (MSE)': np.nan,
                'Validation error (MSE)': np.nan,
                'Time of execution (s)': np.nan
            })
        continue # Skip to the next data size

    # Load data
    df = pd.read_csv(data_file)
    print(f"  Loaded {data_file}")

    # Select features (X) and target (y) using new column names
    X = df[FEATURE_COLUMNS]
    y = df[TARGET_COLUMN]

    # Check target variable stats
    print(f"  '{TARGET_COLUMN}' variable stats for size {size}:")

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=VALIDATION_SPLIT, random_state=RANDOM_STATE
    )

    # Scale Features (X)
    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_val_scaled = x_scaler.transform(X_val)
    n_features = X_train_scaled.shape[1]

    # Scale Target (Y) - Crucial due to potentially large values
    y_scaler = StandardScaler()
    # Reshape y for scaler (expects 2D array)
    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
    y_val_scaled = y_scaler.transform(y_val.values.reshape(-1, 1))
    print("  Features and Target Scaled.")

    # Train Model 1: 1 hidden layer 4 nodes
    print("\nTraining Model: 1 hidden layer 4 nodes")
    model_1L = build_model_simple(n_features, [4], model_name="1L_4N")

    start_time_1L = time.time()
    history_1L = model_1L.fit(
        X_train_scaled, y_train_scaled,
        validation_data=(X_val_scaled, y_val_scaled),
        epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0
    )
    end_time_1L = time.time()
    execution_time_1L = end_time_1L - start_time_1L

    # Evaluate Model 1 (on original scale)
    pred_train_scaled_1L = model_1L.predict(X_train_scaled, verbose=0)
    pred_val_scaled_1L = model_1L.predict(X_val_scaled, verbose=0)
    pred_train_orig_1L = y_scaler.inverse_transform(pred_train_scaled_1L)
    pred_val_orig_1L = y_scaler.inverse_transform(pred_val_scaled_1L)
    train_mse_orig_1L = mean_squared_error(y_train, pred_train_orig_1L)
    val_mse_orig_1L = mean_squared_error(y_val, pred_val_orig_1L)

    print(f"    Training MSE (original scale): {train_mse_orig_1L:.4f}")
    print(f"    Validation MSE (original scale): {val_mse_orig_1L:.4f}")
    print(f"    Execution Time: {execution_time_1L:.2f} seconds")

    # Store results for Model 1
    dl_results.append({
        'Data size': size,
        'Configuration': '1 hidden layer 4 nodes',
        'Training error (MSE)': train_mse_orig_1L,
        'Validation error (MSE)': val_mse_orig_1L,
        'Time of execution (s)': execution_time_1L
    })

    # Train Model 2: 2 hidden layers 4 nodes each
    print("\nTraining Model: 2 hidden layers 4 nodes each")
    model_2L = build_model_simple(n_features, [4, 4], model_name="2L_4N_4N")

    start_time_2L = time.time()
    history_2L = model_2L.fit(
        X_train_scaled, y_train_scaled,
        validation_data=(X_val_scaled, y_val_scaled),
        epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0
    )
    end_time_2L = time.time()
    execution_time_2L = end_time_2L - start_time_2L

    # Evaluate Model 2 (on original scale)
    pred_train_scaled_2L = model_2L.predict(X_train_scaled, verbose=0)
    pred_val_scaled_2L = model_2L.predict(X_val_scaled, verbose=0)
    pred_train_orig_2L = y_scaler.inverse_transform(pred_train_scaled_2L)
    pred_val_orig_2L = y_scaler.inverse_transform(pred_val_scaled_2L)
    train_mse_orig_2L = mean_squared_error(y_train, pred_train_orig_2L)
    val_mse_orig_2L = mean_squared_error(y_val, pred_val_orig_2L)

    print(f"    Training MSE (original scale): {train_mse_orig_2L:.4f}")
    print(f"    Validation MSE (original scale): {val_mse_orig_2L:.4f}")
    print(f"    Execution Time: {execution_time_2L:.2f} seconds")

    # Store results for Model 2
    dl_results.append({
        'Data size': size,
        'Configuration': '2 hidden layers 4 nodes each',
        'Training error (MSE)': train_mse_orig_2L,
        'Validation error (MSE)': val_mse_orig_2L,
        'Time of execution (s)': execution_time_2L
    })



 Data Size: 1000
  Loaded synthetic_data_1k.csv
  'Target' variable stats for size 1000:
  Features and Target Scaled.

Training Model: 1 hidden layer 4 nodes
    Training MSE (original scale): 14552467.0555
    Validation MSE (original scale): 14419139.8553
    Execution Time: 9.25 seconds

Training Model: 2 hidden layers 4 nodes each
    Training MSE (original scale): 73428227.4457
    Validation MSE (original scale): 74458843.2386
    Execution Time: 9.89 seconds

 Data Size: 10000
  Loaded synthetic_data_10k.csv
  'Target' variable stats for size 10000:
  Features and Target Scaled.

Training Model: 1 hidden layer 4 nodes
    Training MSE (original scale): 9462526.3514
    Validation MSE (original scale): 10213397.9534
    Execution Time: 32.30 seconds

Training Model: 2 hidden layers 4 nodes each
    Training MSE (original scale): 6405523.1911
    Validation MSE (original scale): 6441236.3064
    Execution Time: 36.11 seconds

 Data Size: 100000
  Loaded synthetic_data_100k.csv
 